# Model Creation and Training

In [1]:
import pandas as pd
import numpy as np
import logging
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Lambda, Reshape, Concatenate, Add, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, Layer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

logging.basicConfig(filename = "logs.log", format = "%(asctime)s -- %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.INFO)

In [2]:
### Load_Data ###
training_data = pd.read_csv("training_data/10sequence.csv")
training_data.head()
X = training_data.iloc[:,:10]
y = training_data.iloc[:,10]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

logging.info("Dataset Upload successfully")
logging.info(f"X training set shape:{X_train.shape}")
logging.info(f"y training set shape:{y_train.shape}")
logging.info(f"X test set shape:{X_test.shape}")
logging.info(f"y test set shape:{y_test.shape}")


print("Dataset Upload successfully")
print(f"X training set shape:{X_train.shape}")
print(f"y training set shape:{y_train.shape}")
print(f"X test set shape:{X_test.shape}")
print(f"y test set shape:{y_test.shape}")



Dataset Upload successfully
X training set shape:(442920, 10)
y training set shape:(442920,)
X test set shape:(110731, 10)
y test set shape:(110731,)


# Batch Data

In [3]:
### Global Variables ####
sequence_len = 10 # Looking at 24 hours worth of data to determine
epochs = 100
attention_heads = 6
projection_dim = sequence_len
dropout = 0.1
num_transformer_blocks = 8
mlp_units = [2048, 1024, 500, 250, 100, 10]
tranformer_mlp_units = [projection_dim ** 2, projection_dim * 2]


X_train = tf.expand_dims(X_train, axis = 1)

2023-04-03 15:17:51.674745: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
### Convert time to a vector that can be encoded to the features ###
class Time2Vector(tf.keras.layers.Layer):
    def __init__(self, kernal: int = 64):
        super().__init__(trainable = True,  name = "Time2VecLayer")
        self.kernal = kernal - 1
        
    def build(self, input_shape):
        ### Time to Vector Piecewise function ###
        
        self.weights_linear = self.add_weight(shape = (input_shape[1],1), initializer = "uniform", trainable = True, name = "weights_linear")
        
        self.bias_linear = self.add_weight(shape = (input_shape[1],1), initializer = "uniform", trainable = True, name = "bias_linear")
        
        self.weights_periodic = self.add_weight(shape = (input_shape[1], self.kernal), initializer = "uniform", trainable = True, name = "weights_periodic")
        
        self.bias_periodic = self.add_weight(shape = (input_shape[1], self.kernal), initializer = "uniform", trainable = True, name = "bias_periodic")
    
    def call(self, x):
        x = tf.expand_dims(x[:,:,0], axis = -1)
        time_linear = (self.weights_linear * x) + self.bias_linear
        # time_linear = tf.expand_dims(time_linear, axis = -1) #Expand dimensions to concat later
        
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        # time_periodic = tf.expand_dims(time_periodic, axis = -1)
        
        final_product = tf.concat([time_linear, time_periodic], axis = -1)
        # final_product = tf.expand_dims(final_product, axis = 0)
        return final_product
    
###Test Input###    
test_input = tf.random.uniform(shape = (5000, 1, 10), dtype = tf.float32)
exampleTV = Time2Vector(10)(test_input)
exampleTV
        

<tf.Tensor: shape=(5000, 1, 10), dtype=float32, numpy=
array([[[-0.07681987, -0.04576923,  0.04169796, ..., -0.01934507,
         -0.06159167, -0.01717032]],

       [[-0.08798409, -0.05365489,  0.04059355, ..., -0.01711817,
         -0.07210016, -0.01813994]],

       [[-0.08125924, -0.04890528,  0.0412588 , ..., -0.01845957,
         -0.06577118, -0.01755588]],

       ...,

       [[-0.06488314, -0.03733478,  0.04287872, ..., -0.02172596,
         -0.05034851, -0.01613359]],

       [[-0.05202583, -0.02824689,  0.04415048, ..., -0.02429033,
         -0.03823116, -0.01501689]],

       [[-0.06426544, -0.03689824,  0.04293982, ..., -0.02184916,
         -0.04976652, -0.01607994]]], dtype=float32)>

In [5]:

def mlp_block(x, units):
    x = GlobalAveragePooling1D(data_format = "channels_first")(x)
    for unit in units:
        x = Dense(unit, activation = tf.nn.gelu)(x)
        x = Dropout(0.1)(x)
    return x
def transformer_encoder(inputs, attention_heads, projection_dim, dropout):
    ### Layer Normalization / Multihead Attention Layers ###
    x = LayerNormalization(epsilon = 1e-6)(inputs)
    x = MultiHeadAttention(num_heads = attention_heads, key_dim = projection_dim, dropout = dropout)(x,x)
    skip1 = Add()([x, inputs])
    
    ### Feed Forward ###
    x = LayerNormalization(epsilon = 1e-6)(skip1)
    x = mlp_block(x, tranformer_mlp_units)
    skip2 = Add()([x,skip1])
    
    return skip2


def build_model():
    input = Input(shape = X_train.shape[1:]) # (Batch_size, Sequence Length, Number of Features)
    x = Time2Vector(sequence_len)(input)
    x = Concatenate(axis = -1)([input, x]) 
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, attention_heads, projection_dim, dropout)
    x = mlp_block(x, mlp_units)
    output = Dense(1, activation = "relu")(x)
    
    model = Model(inputs = input, outputs = output)
    model.summary()

    return model

def train_model(model):
    optimizer = tf.optimizers.Adam(learning_rate=1e-3, decay = 1e-4)
    checkpoint_path = "/models/"
    model.compile(optimizer=optimizer, 
                  loss = tf.keras.losses.MeanAbsoluteError(), 
                  metrics = [tf.keras.metrics.MeanSquaredError(name = "MSE"), 
                             tf.keras.metrics.RootMeanSquaredError(name = "RMSE"),
                             tf.keras.metrics.MeanAbsoluteError(name = "MAE")])
    
    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = epochs,
        batch_size = X_train.shape[0],
        validation_split = 0.2
        
    )
    
    return history

In [6]:
model = build_model()
training = train_model(model)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 10)]      0           []                               
                                                                                                  
 Time2VecLayer (Time2Vector)    (None, 1, 10)        20          ['input_1[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 1, 20)        0           ['input_1[0][0]',                
                                                                  'Time2VecLayer[0][0]']          
                                                                                                  
 layer_normalization (LayerNorm  (None, 1, 20)       40          ['concatenate[0][0]']        